# **TP1 - Exercicio 1- Grupo 11**

Bruno Miguel Fernandes Araújo - A97509

Tiago Emanuel Lemos Teixeira - A97666


1. Pretende-se construir o horário semanal de aulas de uma turma.
    1. Existe um conjunto de salas $S$ classificadas em “grandes” e “pequenas”.
    2. O tempo do horário está organizado em “slots” de uma hora. O total do tempo disponível é 5 horas de manhã e 5 horas às tarde.
    3. Existe um conjunto $D\,$ de disciplinas.  Cada disciplina tem um atributo $dur\in\{1,2\}$ que classifica a duração de cada sessão (um ou dois “slots”) , um atributo $a\in\{2,3\}$ que define o número de sessões semanais e um atributo $\,tam\in\{0,1\}\,$ que diz se a sessão necessita de uma sala grande ou não.
     d. Existe um conjunto $P$ de professores. Cada professor tem associado um conjunto  $h$ das disciplinas que está habilitado a lecionar.
     e. O horário está organizado em sessões concorrentes onde cada sessão é definido por uma disciplina desce que salas e professores verifiquem as seguintes restrições.
        1. Para cada disciplina todas as aulas decorrem na mesma sala e com o mesmo professor.
        2. O número total de horas lecionadas por cada professor está num intervalo de $\pm 20\%\,$do número médio de horas lecionadas pela totalidade dos professores.
        3. Nenhuma sala pode ser ocupada simultaneamente por mais do que uma aula e nenhum professor pode lecionar simultaneamente mais do que uma aula.
        4. Em cada disciplina, cada aula é lecionada por um professor habilitado para essa disciplina e ocorre numa sala de tamanho apropriado à disciplina.

# **Criação de matrizes**
Criamos de um solver chamado **horario** e 4 matrizes:

**X**: Matriz de alocação

**SD**: Matriz Salas x Disciplinas

**PD**: Matriz Professores x Disciplinas

**SL**: Matriz Slots x Disciplinas que será utilizada para a otimização do problema

In [5]:
from ortools.linear_solver import pywraplp

# Create the LP solver with the SCIP backend.

horario = pywraplp.Solver.CreateSolver("SCIP")
error  = pywraplp.Solver.INFEASIBLE

In [6]:
#Exemplos


Slots = 10
Dias = 5

#Exemplo 1

#Prof = 5                 
#Professores={1:[1,2,3], 2:[3], 3:[1], 4:[1,2], 5:[2,3]}

#Sal = 3                
#Salas=[1,0,1]

#Disc = 3               
#Disciplinas={1:(1,2,1),2:(2,3,0),3:(2,2,1)}

#Exemplo 2

#Prof = 5                 
#Professores={1:[1,2,3], 2:[3], 3:[4,5] }

#Sal = 7                
#Salas=[1,0,1,1,1,0,0]

#Disc = 3               
#Disciplinas={1:(1,2,1),2:(2,3,0),3:(2,2,1),4:(2,2,0),5:(2,2,1)}

#Exemplo 3

Prof = 3
Professores={1:[1], 2:[2], 3:[3] }

Sal = 3                
Salas=[1,1,1]

Disc = 3               
Disciplinas={1:(1,2,1),2:(1,2,1),3:(1,2,1)}


#Matriz de alocação


X = {}

for dia in range(Dias):
    X[dia] = {}
    for sl in range(Slots):
        X[dia][sl] = {}
        for s in range(len(Salas)):
            X[dia][sl][s] = {}
            for p in range(len(Professores)):
                X[dia][sl][s][p] = {}
                for d in range(len(Disciplinas)):
                    X[dia][sl][s][p][d] = horario.BoolVar(f'[{dia}][{sl}][{s}][{p}][{d}]')

#Matriz Salas x Disciplinas
SD = {}
for s in range(len(Salas)):
    SD[s] = {}
    for d in range(len(Disciplinas)):
        SD[s][d] = horario.BoolVar(f'[{s}][{d}]')
        
#Matriz Professores x Disciplinas
PD = {}
for p in range(len(Professores)):
    PD[p] = {} 
    for d in range(len(Disciplinas)):
        PD[p][d] = horario.BoolVar(f'[{p}][{d}]')
        
#Matriz Slots x Disciplinas
SL = {}
for s in range(Slots):
    SL[s] = {} 
    for d in range(len(Disciplinas)):
        SL[s][d] = horario.BoolVar(f'[{s}][{d}]')



# **Restrições**

1. Cada disciplina tem que ter ss*dur slots durante a semana (ss - sessões semanais; dur - duraçao de uma aula)

$$ \forall_{d<D}, \sum_{dia<Dias,hora<Slots,s<S,p<P} X[dia][hora][s][p][d] = (ss*dur)$$

In [7]:
#Cada disciplina tem que ter ss*dur slots durante a semana (ss - sessões semanais; dur - duraçao de uma aula)


for d in range(len(Disciplinas)):
    dur = Disciplinas[d+1][0]
    ss = Disciplinas[d+1][1]
    horario.Add(horario.Sum(X[dia][hora][s][p][d] for dia in range(Dias) 
                                                  for hora in range(Slots) 
                                                  for s in range(len(Salas)) 
                                                  for p in range(len(Professores)))==(ss*dur))

2. Aulas com Disciplinas (ou Professores ou Salas) iguais encontra-se num determinado slot uma ou nenhuma vez


  2.1. Disciplinas    
$$ \forall_{dia<Dias} \forall_{hora<Slots} \forall_{d<D}, \sum_{s<S,p<P} X[dia][hora][s][p][d] \le 1$$

In [8]:
#Uma disciplina encontra-se num determinado slot uma ou nenhuma vez
for dia in range(Dias):
    for hora in range(Slots):
        for d in range(len(Disciplinas)):
            horario.Add(horario.Sum(X[dia][hora][s][p][d] for s in range(len(Salas)) for p in range(len(Professores)))<=1)

2.2. Professores
    $$ \forall_{dia<Dias} \forall_{hora<Slots} \forall_{p<P}, \sum_{s<S,d<D} X[dia][hora][s][p][d] \le 1$$

In [9]:
#Um professor encontra-se num determinado slot uma ou nenhuma vez
for dia in range(Dias):
    for hora in range(Slots):
        for p in range(len(Professores)):
            horario.Add(horario.Sum(X[dia][hora][s][p][d] for s in range(len(Salas)) for d in range(len(Disciplinas)))<=1)

2.3. Salas
    $$ \forall_{dia<Dias}, \forall_{hora<Slots}, \forall_{s<S}, \sum_{p<P,d<D} X[dia][hora][s][p][d] \le 1$$

In [10]:
#Uma sala encontra-se num determinado slot uma ou nenhuma vez
for dia in range(Dias):
    for hora in range(Slots):
        for s in range(len(Salas)):
            horario.Add(horario.Sum(X[dia][hora][s][p][d] for d in range(len(Disciplinas)) for p in range(len(Professores)))<=1)

**3. Restrições sobre Professores**

3.1. Cada Disciplina tem um unico professor

$$\forall_{d<D}, \sum_{p<P} PD[p][d] = 1$$

In [11]:
for d in range(len(Disciplinas)):
    horario.Add(horario.Sum(PD[p][d] for p in range(len(Professores))) == 1)

In [12]:
for dia in range(Dias):
    for hora in range(Slots):
        for p in range(len(Professores)):
            for s in range(len(Salas)):
                for d in range(len(Disciplinas)): 
                    horario.Add(PD[p][d] >= X[dia][hora][s][p][d])

3.2. Alocaçao de professores para as disciplinas que está habilitado

$$ \forall_{dia<Dias} \forall_{hora<Slots}, \forall_{s<S}, \forall_{d<D}, \forall_{p<P}, (d+1) \notin Professores[p+1] \rightarrow X[dia][hora][s][p][d] \le 1$$

In [13]:
for dia in range(Dias):
    for hora in range(Slots):
        for s in range(len(Salas)):
            for d in range(len(Disciplinas)):
                for p in range(len(Professores)):
                    if d+1 in Professores[p+1]:
                        horario.Add(X[dia][hora][s][p][d] <= 1)
                    else:
                        horario.Add(X[dia][hora][s][p][d] == 0) 


3.3. O número total de horas lecionadas por cada professor está num intervalo de 20% do número médio de horas lecionadas pela totalidade dos professores $$(Disciplinas[d+1][0] = dur; Disciplinas[d+1][1] = a)$$

$$\forall_{p<P}, (media*1.2) \ge (\sum_{d<D} PD[p][d] * dur * a) \ge (media*0.8)$$

In [14]:

#s_Total = 0

#for d1 in range(len(Disciplinas)):
#    s_Total += Disciplinas[d1+1][0] * Disciplinas[d1+1][1]

#media = s_Total/len(Professores)

#for p in range(len(Professores)):
#    horario.Add((media*1.2)>=horario.Sum(PD[p][d] * Disciplinas[d+1][0] * Disciplinas[d+1][1] for d in range(len(Disciplinas)))>=(media*0.8))

**4. Restrições sobre Salas**

4.1. Cada Disciplina tem uma unica sala

$$\forall_{d<D}, \sum_{s<S} SD[s][d] = 1$$

In [15]:
for d in range(len(Disciplinas)):
    horario.Add(horario.Sum(SD[s][d] for s in range(len(Salas))) == 1)

4.2. Uma determinada sala tem que estar associada de pelo menos uma certa disciplina

$$ \forall_{dia<Dias} \forall_{hora<Slots}, \forall_{s<S}, \forall_{d<D}, \forall_{p<P}, SD[s][d] \le X[dia][hora][s][p][d]$$

In [16]:
for dia in range(Dias):
    for hora in range(Slots):
        for p in range(len(Professores)):
            for s in range(len(Salas)):
                for d in range(len(Disciplinas)): 
                    horario.Add(SD[s][d] >= X[dia][hora][s][p][d])

4.3. Alocaçao de salas de tamanho adequado para uma dada disciplina $$(Disciplinas[d+1][2] = tam)$$

$$ \forall_{dia<Dias} \forall_{hora<Slots}, \forall_{s<S}, \forall_{d<D}, \forall_{p<P}, tam = Salas[s] \rightarrow X[dia][hora][s][p][d] \le 1 $$

In [17]:
for dia in range(Dias):
    for hora in range(Slots):
        for p in range(len(Professores)):
            for d in range(len(Disciplinas)):
                for s in range(len(Salas)):
                    if Disciplinas[d+1][2]==Salas[s]:
                        horario.Add(X[dia][hora][s][p][d] <= 1)
                    else:
                        horario.Add(X[dia][hora][s][p][d] == 0)

# **Imprimir o horário**

In [18]:
#Imprimir

status = horario.Solve()
if status == pywraplp.Solver.OPTIMAL:
 
 for dia in range(Dias):
     print(f"Dia {dia + 1}:")
     for slot in range(Slots):
         if slot == 0:
            print(f"\nManhã:\n")
         if slot == 5:
            print(f"\nTarde:\n")
         print(f"Slot {slot + 1}: ")
         for s in range(len(Salas)):
             for p in range(len(Professores)):
                 for d in range(len(Disciplinas)):
                        value = X[dia][slot][s][p][d].solution_value()
                        if value == 1:
                            print(f"Room {s + 1}, Discipline {d + 1}, Professor {p + 1}", end=" | ")
         print("\n")  
     print() 
        
else:
  print("Não foi encontrada solução")

Dia 1:

Manhã:

Slot 1: 


Slot 2: 


Slot 3: 


Slot 4: 


Slot 5: 



Tarde:

Slot 6: 


Slot 7: 


Slot 8: 
Room 2, Discipline 1, Professor 1 | 

Slot 9: 


Slot 10: 
Room 1, Discipline 3, Professor 3 | 


Dia 2:

Manhã:

Slot 1: 


Slot 2: 


Slot 3: 


Slot 4: 


Slot 5: 



Tarde:

Slot 6: 


Slot 7: 


Slot 8: 


Slot 9: 


Slot 10: 



Dia 3:

Manhã:

Slot 1: 


Slot 2: 


Slot 3: 


Slot 4: 


Slot 5: 



Tarde:

Slot 6: 


Slot 7: 


Slot 8: 
Room 2, Discipline 1, Professor 1 | 

Slot 9: 
Room 1, Discipline 2, Professor 2 | 

Slot 10: 



Dia 4:

Manhã:

Slot 1: 


Slot 2: 


Slot 3: 


Slot 4: 


Slot 5: 



Tarde:

Slot 6: 


Slot 7: 


Slot 8: 


Slot 9: 


Slot 10: 
Room 1, Discipline 2, Professor 2 | 


Dia 5:

Manhã:

Slot 1: 


Slot 2: 


Slot 3: 


Slot 4: 


Slot 5: 
Room 1, Discipline 3, Professor 3 | 


Tarde:

Slot 6: 


Slot 7: 


Slot 8: 


Slot 9: 


Slot 10: 





# **Restrições Incompletas**



In [18]:
#maximize o número de partes de dia (manhãs ou tardes) que estão livres de qualquer aula.
for hora in range(,5)
        horario.Maximize(horario.Sum(SL[hora][d] for d in range(len(Disciplinas))) == 0)

In [19]:
#Cada sessão tem um numero de horas consecutivas
#
#    

In [16]:
#Cada sessão tem um numero de horas consecutivas

for d in range(len(Disciplinas)):
    if Disciplinas[d+1][0] == 1:
        for dia in range(Dias):
            for s in range(len(Salas)):
                for p in range(len(Professores)):
                    horario.Add(horario.Sum(X[dia][hora][s][p][d] for hora in range(Slots)) <= 1)
    if Disciplinas[d+1][0] == 2:
        for dia in range(Dias):
            for s in range(len(Salas)):
                for p in range(len(Professores)):
                    for hora in range(Slots-1):
                        if (hora != 0 and hora != 4):
                            horario.Add(X[dia][hora][s][d][p] <= horario.Sum(X[dia][h1][s][d][p] for h1 in range(hora-1, hora+2, 2)))
